In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet
import re 
from spellchecker import SpellChecker
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Pre-Running

In [3]:
# Building a list of Keywords
list_words=['hello','analyze', 'quit']
list_syn={}
for word in list_words:
    synonyms=[]

    # Create synonyms for each Keyword
    for syn in wordnet.synsets(word):
        for lem in syn.lemmas():

            # Remove any special characters from synonym strings
            lem_name = re.sub('[^a-zA-Z0-9 \n\.]', ' ', lem.name())
            synonyms.append(lem_name)
    list_syn[word]=set(synonyms)


print (list_syn['analyze'])

{'dissect', 'study', 'canvas', 'take apart', 'psychoanalyse', 'canvass', 'analyse', 'psychoanalyze', 'analyze', 'break down', 'examine'}


In [12]:
# English Stopwords for clean_user_input function
stop_words = set(stopwords.words("english"))
# Spellchecker
spell = SpellChecker()
# Lemmatzier
lemmatizer = WordNetLemmatizer()


def clean_user_input(user_input):
    # Lowercase user input
    user_input_lower = user_input.lower()

    # Remove punctuation
    user_input_punct = user_input_lower.translate(str.maketrans('', '', string.punctuation))
    
    # Spellchecker
    words = user_input_punct.split()
    correct_words = [spell.correction(word) for word in words]

    # Remove Stopwords
    filtered_words = [word for word in correct_words if word not in stop_words]

    # Lemmatization
    base_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # Join words into sentence              
    lower_case_response = ' '.join(base_words)
    
    return lower_case_response



def generate_response(user_input):
    patterns = {
        r'(?i)({}).*'.format('|'.join(list_syn['hello'])): "How can I help you?",
        r'(?i)analyze.*': "{}".format(clean_user_input(user_input)),
        r'(?i)quite.*': "See You!"
    }
    for pattern, response in patterns.items():
        match = re.match(pattern, user_input)
        if match:
            return response
    return "I'm sorry, but I'm not sure I understand."



def chat():
    print("Hi")
    while True:
        user_input = input("You: ")

        if user_input.lower() == 'analyze':
            lower_case_response = clean_user_input(user_input)
            print(lower_case_response)

        # Here: Call Function for Logistic Regression or Naiv Bayes

        if user_input.lower() == 'quit':
            print("Goodbye!")
            break

        response = generate_response(user_input)
        print("Mr. C-Bot:", response)

# Example usage
chat()


Hi


Mr. C-Bot: How can I help you?
Mr. C-Bot: How can I help you?
Mr. C-Bot: analyze plot film predictable saw twist coming mile away
